In [18]:
# ensure that the imported .py file will get auto imported and updated whenever there is a change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# !pip install PyMuPDF

In [19]:

import nltk, os, re
from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from nltk.corpus import stopwords
import pandas as pd

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import processing_pdf
pd.options.display.max_colwidth = 1000


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:

project_path = os.getcwd()
project_data_path = project_path + "/data"
project_processed_data_path = project_path + "/processed"
if not os.path.exists(project_data_path):
    os.makedirs(project_data_path)
if not os.path.exists(project_processed_data_path):
    os.makedirs(project_processed_data_path)

In [21]:
# pdf_file =  "An Empirical Survey on Long Document Summarization.pdf"
pdf_file = "1901.00009v1.pdf"
# pdf_file = "1901.00936v3.pdf"
# pdf_file = "an image is worth 16 by 16 words.pdf"
doc, total_text, total_pages = processing_pdf.open_file(project_data_path + "/" + pdf_file)
table_of_content = doc.get_toc()
print("Auto generated table of content:")
display(table_of_content)

# some papers have not table of content
if len(table_of_content) == 0:
    print("The paper has not table of content. Need to use regular expression to map table of content.")
    res = processing_pdf.auto_find_toc(doc)
    print("Not satified the generated result or want to adjust? Build your own table of content by using this template:")
    display(res)



Auto generated table of content:


[[1, '1 Introduction', 1],
 [1, '2 Observations and Data reduction', 2],
 [1, '3 Variability Analysis', 3],
 [2, '3.1 Cross-matches to external catalogs', 4],
 [2, '3.2 Variability cutoffs', 4],
 [2, '3.3 Variability Classification', 4],
 [2, '3.4 Blending Corrections', 6],
 [1, '4 Results', 6],
 [1, '5 Conclusions', 8]]

In [22]:
#uncomment this list to customize table-of-content
# table_of_content = [[1, 'I. INTRODUCTION', 1],
#  [1, 'II. SFC BASED ON IPV6 SEGMENT ROUTING', 2],
#  [1, 'III. DESIGN OF THE SRV6 PROXY', 4],
#  [2, 'A. General Concepts and State-of-the-art', 4],
#  [2, 'B. SRNKv1', 5],
#  [2, 'C. SRNKv2', 7],
#  [2, 'D. Implementation of other SR proxy types', 8],
#  [1, 'IV. TESTING ENVIRONMENT', 8],
#  [1, 'V. PERFORMANCE ANALYSIS', 9],
#  [1, 'VII. CONCLUSIONS', 11]]

# separate content into sections
processing_pdf.clear_processed_folder(project_processed_data_path)
title, authors, other_info, abstract = processing_pdf.find_meta_data(doc, table_of_content)
ds, json_dict = processing_pdf.separate_content(total_text, table_of_content)
processing_pdf.save_dataframe(ds, json_dict, project_processed_data_path,  pdf_file.rsplit(".", 1)[0])
# extract images
processing_pdf.find_images(doc, table_of_content, total_pages, project_processed_data_path)

# display(ds)
print(f"\nTitle: {title}")
print(f"\nAuthors:{authors}")
print(f"\nOther info: {other_info}")
print(f"\nAbstract:{abstract}")


starting looking for all the sections according to the provided section title info...
save the dataframe to d:\Start_Dec_19_2023\application\NUS Course\Semester_1\Neural Network and Deep Learning\PDF_Conversion/processed/1901.00009v1.csv
save the dataframe to d:\Start_Dec_19_2023\application\NUS Course\Semester_1\Neural Network and Deep Learning\PDF_Conversion/processed/1901.00009v1.json

Title: The ASAS-SN Catalog of Variable Stars III: Variables inthe Southern TESS Continuous Viewing Zone

Authors:T. Jayasinghe1,2⋆, K. Z. Stanek1,2, C. S. Kochanek1,2, B. J. Shappee3,T. W. -S. Holoien4, Todd A. Thompson1,2,5, J. L. Prieto6,7, Subo Dong8, M. Pawlak9,O. Pejcha9, J. V. Shields1, G. Pojmanski10, S. Otero11, N. Hurst12, C. A. Britt12,D. Will1,12

Other info: 1Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA2Center for Cosmology and Astroparticle Physics, The Ohio State University, 191 W. WoodruﬀAvenue, Columbus, OH 43210, USA3Institute for A

In [ ]:
import json

# open json file
with open(project_processed_data_path + "/an image is worth 16 by 16 words.json") as f:
    data = json.load(f)
    for item in data:
        display(item)